In [ ]:
import pandas as pd

df = pd.read_csv('https://drive.google.com/uc?id=15RfMD9lNkpS3cVN7j3_dsJKZ8_5RJG5z')
df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [ ]:
import tensorflow as tf

# Function yang dapat melakukan proses windowing
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1) # mengubah dimensi series dengan menambahkan satu dimensi baru di posisi terakhir (axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series) # membuat tensorflow menjadi sebuah series.
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # membagi dataset menjadi jendela jendela berukuran window_size + 1 dengan jarak 1 (shift 1) antara jendela jendela tersebut.
    ds = ds.flat_map(lambda w: w.batch(window_size + 1)) # mengubah setiap window menjadi sebuah batch dengan ukuran window size + 1.
    ds = ds.shuffle(shuffle_buffer) # mengacak dataset dengan menggunakan buffer ukuran shuffle_buffer.
    ds = ds.map(lambda w: (w[:-1], w[-1:])) # memtekan setiap batch menjadi sepasang data input dan target.
    return ds.batch(batch_size).prefetch(1) # mengelompokkan batch-batch dataset menjadi batch dengan ukuran batch_size, dan menambahkan prefetching ke dataset.

menentukan fitur yang akan digunakan untuk melakukan windowing dataset.

In [ ]:
dates = df['date'].values
temp = df['meantemp'].values

In [ ]:
train_set = windowed_dataset(temp, window_size=2, batch_size=3, shuffle_buffer=1000)

Catatan:

Dalam konteks time series, meskipun data telah diurutjan berdasarkan waktu, pengacakan tetap diperlukan. hal ini karena saat menggunakan windows untuk melatih model, kita ingin model hanya belajar dari urutan yang sama setiap epoch. pengacakan memastikan bahwa setiap epoch model melihat sampel sampel dalam urutan yang berbeda, sehingga model dapat mempelajari pola piola yang lebih umum dan tidak hanya mengingat urutan tertentu. semakin besar nilai shuffle_buffer, semakin banyak elemen yang akan diambil secara acakm dan semakin besar variasi yang mungkin dihasilkan dalam setiap epoch. namun, menggunakan nilai yang terlalu besar juga dapat memakan banyak memori karena bugger tersebut harus menampung elemen-elemen dataset.

In [ ]:
for data in train_set:
    print(data)
    break

(<tf.Tensor: shape=(3, 2, 1), dtype=float64, numpy=
array([[[34.875],
        [35.25 ]],

       [[11.   ],
        [12.5  ]],

       [[35.25 ],
        [36.875]]])>, <tf.Tensor: shape=(3, 1, 1), dtype=float64, numpy=
array([[[36.875]],

       [[12.875]],

       [[37.5  ]]])>)


Seperti yang dapat kita lihat pada output di atas, dataset yang kita miliki berisikan masing-masing dua buah data. Selain itu, karena kita menentukan batch_size dengan nilai tiga, dalam satu batch hanya berisikan tiga buah window. Pada array pertama menunjukkan fitur input, sedangkan array kedua menunjukkan fitur target. Keduanya sangat mudah dimengerti, bukan?